<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路分析

* 節點
* 連線
  * 無向(undirected)連線
  * 有向(directed)連線

## 中心度分析
* 分支中心度 (degree centrality)
* 居間中心度 (betweenness centrality)
* 本徵向量中心度 (eigenvector centrality)

### 圖 22-1 DataScienceter 的朋友的關係網路
![22-1](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-1.png?raw=true)

建構構成網路的使用者

In [ ]:
users = [
  {"id":0,"name":"Hero"},
  {"id":1,"name":"Dunn"},
  {"id":2,"name":"Sue"},
  {"id":3,"name":"Chi"},
  {"id":4,"name":"Thor"},
  {"id":5,"name":"Clive"},
  {"id":6,"name":"Hicks"},
  {"id":7,"name":"Devin"},
  {"id":8,"name":"Kate"},
  {"id":9,"name":"Klein"}
     ]

以及彼此間的朋友關係

In [ ]:
friendship_pairs = [
         (0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
         (4,5),(5,6),(5,7),(6,8),(7,8),(8,9)
         ]

# <1> 分支中心度 (degree centrality)
易於計算，能找出網路中擁有最多連線的節點

In [ ]:
friendships = {user["id"]:[] for user in users}
for i,j in friendship_pairs:
  friendships[i].append(i)
  friendships[i].append(j)

加總每個人的朋友數量

In [ ]:
def number_of_friends(user):
  """_user_ 有多少朋友?"""
  user_id = user["id"]
  friend_ids = friendships[user_id]
  return len(friend_ids)

total_connections = sum(number_of_friends(user) for user in users)

再除以使用者總人數

In [ ]:
num_users = len(users)
avg_connections = total_connections/num_users

取得每個使用者的連接使用者數目資訊

In [ ]:
num_friends_by_id = [(user["id"],number_of_friends(user)) for user in users]

num_friends_by_id.sort(
    key = lambda id_and_friends:id_and_friends[1], # 根據朋友數量
    reverse = True                 # 從最多排序到最少
)
print(num_friends_by_id)

[(0, 4), (1, 4), (5, 4), (2, 2), (3, 2), (4, 2), (6, 2), (7, 2), (8, 2), (9, 0)]


依據節點的連線個數，建立列表

In [ ]:
x = list(set([count[1] for count in num_friends_by_id]))
x.sort(reverse=True)
y = {number:[count[0] for count in num_friends_by_id if count[1]==number] for number in x}

print(x)
print(y)

[4, 2, 0]
{4: [0, 1, 5], 2: [2, 3, 4, 6, 7, 8], 0: [9]}


### 圖 1-2 DataSciencter的朋友關係網路，並以不同大小來呈現不同的分支之中心度
![22-2](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-2.png?raw=true)

# <2> 居間中心度 (betweenness centrality)
找出網路中的**關鍵聯繫者**  
識別出常出現在任兩者之間最短路徑的人  
  
**例如**：欲找出節點i的居間中心度  
針對其他兩兩成對的節點j和節點k，計算出兩點之間所有最短路徑包含節點i的比例，  
最後再將各組的比例值全部加總起來


In [ ]:
from typing import NamedTuple

class User(NamedTuple):
  id:int
  name:str

建構構成網路的使用者

In [ ]:
users = [
      User(0,"Hero"),
      User(1,"Dunn"),
      User(2,"Sue"),
      User(3,"Chi"),
      User(4,"Thor"),
      User(5,"Clive"),
      User(6,"Hicks"),
      User(7,"Devin"),
      User(8,"Kate"),
      User(9,"Klein")
     ]

以及彼此間的朋友關係

In [ ]:
friend_pairs = [
         (0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
         (4,5),(5,6),(5,7),(6,8),(7,8),(8,9)
         ]

使用dict表示朋友關係

In [ ]:
from typing import Dict,List

# 用來追蹤朋友關係的型別別名
Frinedships = Dict[int,List[int]]

friendships:Frinedships = {user.id:[] for user in users}

for i,j in friend_pairs:
  friendships[i].append(j)
  friendships[j].append(i)

assert friendships[4] == [3,5]
assert friendships[8] == [6,7,9]

# 建構**廣度優先搜尋演算法(BFS,Breadth-first search)**
1. 我們的目標是製作出一個以 from user為輸入項的函式,這個函式可以找出從這個 from_user 連往所有其他使用者的全部最短路徑。

2. 我們會用使用者編號組成 list 列表,來表示一條路徑。由於每條路徑都是以 from_user 做為起點,所以在列表中就不包含 from_user 的編號了。這麼一來,這個代表路徑的列表其中元素的數量,也就等於路徑本身的長度。

3. 我們會用到個叫做shortest_paths_to 的dict字典,其值為使用者編號,而相應的值則是路徑未端為該使用者的全部最短路徑列表,如果只有唯一一條最短路徑,列表中就只會包含這條路徑。如果有好幾條最短路徑,列表中則會包含所有這些最短路徑。

4. 我們也會用到一個叫做frontier的佇列(queue),裡頭會根據我們打算進行探索的順序,存放我們打算探索的使用者資料。我們是以(**前一使用者**,**使用者**)這樣的成對形式,把使用者資料存放在這個件列中如此一來,我們就可以運用前一使用者的資訊,求取出使用者相應的結果一開始我們會把from_user與他所有的朋友一一配對,然後放進這個作列中,以做為初始設定(我之前並沒有討論過「作 列」這種資料結構,它特別針對「在未端添加資料」與「從前端移出資料」這兩種 操作,做了些最佳化的設計。Python 的 collections.deque 實作了這個資料結構: 而且實作出來的其實是個雙向 [double-ended]佇列)。

5. 我們在網路中進行探索時,只要遇到還不知道其最短路徑的新相鄰節點,就把它加到佇列的末端,以便在稍後進行探索,而當前正在進行探索的使用著,就成為它的prev_user(前一使用者)。

6. 從佇列中取出某個使用者時,就算之前從沒遇過這位使用者,我們還是知道至少有一條從 from_user連往它的最短路徑,因為只要在每條連往 prev_user 的最短路徑後面再加一步就是了。

7. 從佇列中取出某個使用者時,如果我們之前遇過這位使用者,那我們所找到的新路徑,如果不是另條最短路徑(這樣的話還是要把它加進去),要不就是另一條比較長的路徑(這樣就不用再加進去了)。

8. 如果佇列中已經沒有更多使用者的資料,也就等於我們已經探索了整個網路(或者至少可以說,探索範圍已經涵蓋起始使用者可抵達的部分),而工作也就算是完成了。


In [ ]:
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id:int,friendships:Frinedships) ->Dict[int,List[Path]]:
  # 一個dict字典：
  # {使用者編號:「所有」其他使用者聯網此使用者的最短編號}
  shortest_paths_to:Dict[int,List[Path]] = {from_user_id:[[]]}
  # 存放(prev_user,user)的佇列
  # 一開始將(from_user,from_user 的朋友) 全放進佇列中
  frontier = deque((from_user_id,friend_id) for friend_id in friendships[from_user_id])

  # 重複這個操作直到 frontier 佇列為空
  while frontier:
    # 自佇列前端取出下一對成對的資料
    prev_user_id,user_id = frontier.popleft()

    # 至此，已經能知道一些連往 prev_user 的最短路徑 
    paths_to_prev_user = shortest_paths_to[prev_user_id]
    new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

    # 可能已經知道一條通往 user_id 的最短路徑
    old_paths_to_user = shortest_paths_to.get(user_id,[])

    # 判定目前的路徑長度
    if old_paths_to_user:
      min_path_length = len(old_paths_to_user[0])
    else:
      min_path_length = float("inf")
    
    # 新路徑不能重複且長度需短於舊路徑
    new_paths_to_user = [path for path in new_paths_to_user if len(path) <= min_path_length and path not in old_paths_to_user]

    shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

    # 將沒見過的節點放置到 frontier 之中
    frontier.extend((user_id,friend_id) for friend_id in friendships[user_id] if friend_id not in shortest_paths_to)
  
  return shortest_paths_to

**以演算法角度展示計算結果**  
  
| Select|0|1|2|3|4|5|6|7|8|9|
| ---- |-|-|-|-|-|-|-|-|-|-|
| 0 |0|①|∞|∞|∞|∞|∞|∞|∞|∞|
| 1 |0|1|①|2|∞|∞|∞|∞|∞|∞|
| 2 |0|1|1|②|3|∞|∞|∞|∞|∞|
| 3 |0|1|1|2|③|4|∞|∞|∞|∞|
| 4 |0|1|1|2|3|④|5|5|∞|∞|
| 5 |0|1|1|2|3|4|⑤|5|∞|∞|
| 6 |0|1|1|2|3|4|5|⑤|6|∞|
| 7 |0|1|1|2|3|4|5|5|⑥|∞|
| 8 |0|1|1|2|3|4|5|5|6|⑦|
| 9 |0|1|1|2|3|4|5|5|6|7|


計算所有的最短路徑

In [ ]:
# 針對每個 from_user ，列出通往每個 to_user 的所有最短路徑
shortest_paths = {user.id:shortest_paths_from(user.id,friendships) for user in users}
print(shortest_paths)

{0: {0: [[]], 1: [[1]], 2: [[2]], 3: [[1, 3], [2, 3]], 4: [[1, 3, 4], [2, 3, 4]], 5: [[1, 3, 4, 5], [2, 3, 4, 5]], 6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]], 7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]], 8: [[1, 3, 4, 5, 6, 8], [2, 3, 4, 5, 6, 8], [1, 3, 4, 5, 7, 8], [2, 3, 4, 5, 7, 8]], 9: [[1, 3, 4, 5, 6, 8, 9], [2, 3, 4, 5, 6, 8, 9], [1, 3, 4, 5, 7, 8, 9], [2, 3, 4, 5, 7, 8, 9]]}, 1: {1: [[]], 0: [[0]], 2: [[2]], 3: [[3]], 4: [[3, 4]], 5: [[3, 4, 5]], 6: [[3, 4, 5, 6]], 7: [[3, 4, 5, 7]], 8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]], 9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]}, 2: {2: [[]], 0: [[0]], 1: [[1]], 3: [[3]], 4: [[3, 4]], 5: [[3, 4, 5]], 6: [[3, 4, 5, 6]], 7: [[3, 4, 5, 7]], 8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]], 9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]}, 3: {3: [[]], 1: [[1]], 2: [[2]], 4: [[4]], 0: [[1, 0], [2, 0]], 5: [[4, 5]], 6: [[4, 5, 6]], 7: [[4, 5, 7]], 8: [[4, 5, 6, 8], [4, 5, 7, 8]], 9: [[4, 5, 6, 8, 9], [4, 5, 7, 8, 9]]}, 4: {4: [[]], 3: [[3]], 5: [[5]], 1: [[3, 1]], 2: [[3

## 計算**居間中心度**
**已知**：每組節點i與j之間各有n條最短路徑  
**動作**：在每條最短路徑上，為每個節點增加1/n的中心度

In [ ]:
betweenness_centrality = {user.id:0.0 for user in users}

for source in users:
  for target_id,paths in shortest_paths[source.id].items():
    if source.id < target_id: # 不要重複計算
      num_paths = len(paths) # 計算共有幾條最短路徑
      contrib = 1/num_paths  # 對於中心度的貢獻度
      for path in paths:
        for between_id in path:
          if between_id not in [source.id,target_id]:
            betweenness_centrality[between_id] += contrib

print(betweenness_centrality)

{0: 0.0, 1: 3.5, 2: 3.5, 3: 18.0, 4: 20.0, 5: 20.5, 6: 6.0, 7: 6.0, 8: 8.5, 9: 0.0}


### 圖 22-2 DataSciencter的朋友關係網路，並以不同大小來呈現不同的居間中心度

![pic 22-2](https://lh3.googleusercontent.com/iIh2hCnEdh7tVEnd9h8cI5u1kW-MkqQ8qE7FQRBBrXlVQXHUJAtXjUUGSW_-ru4lIM-bL5NYJ7TlYHiaO9fhSFT1nEHRK30IeJcKs-0IgFiYq3gJ6Nx-DTpHHCkjRdQRUXmR7lwUuo8LKqviZ5KKQIoWI7_ssCs8-DLgqNmB3zMEa3JVjvnDDvOTDdY6zESo1-e_o38xg2QYDGkBgbYe-qxC6qdzGW8YoUF0UD3gPEWR3Bb38hPqwwU6Aui0SCG1trII2GPAh8vxEBA-m9Qz45GlvH3hCIh50jnusAQ_mQF9YPzSLqq-WbwEN4cZo4iNwbbvBbvwPWhdRUJkI5BcNRpvqO_U58mM1BFrIC9_KNa47-72dJAQsO4YeX-tycyn-pT72VNEPKlTNEMLZDvvA4yH2Zs9_ltNwIUNRXbnJgcfXh1Ai_BTD8OFIXZ8sHjNJL9XLXyaQIv8SNpEZTJIQNA2ecg4Pv5IxbGuOn_0aln9JOmbNrXKpHxYyvAYyPeCDXbwY3u0RDPkW-4QysFmfEyVeY4sJHLMEjHdKMpHil-iE6GAJE6MLrAtkI2GwjZM6aPHwQerihZfOZMzxc4sS5yfZkUWZAm7OtNnT-uIcz-JMnADVphJ1oFE2nrbrF-uIrKuG3Pi-fGOw1rk9k_FsRC4mS5YsSb1iCP5cARUhlOh0edlqNm9_ZZ5pGg63Q=w346-h99-no?authuser=0)

# <3> 緊密中心度(closeness certrality)
1. 計算每個使用者的**疏遠度(farness)**
2. 將路線加總起來即可

In [ ]:
def farness(user_id:int) ->float:
  """與所有其他使用者之間最短路徑的長度總和"""
  return sum(len(paths[0]) for paths in shortest_paths[user_id].values())

計算緊密中心度

In [ ]:
clossness_centrality = {user.id:1/farness(user.id) for user in users}
print(clossness_centrality)

{0: 0.029411764705882353, 1: 0.037037037037037035, 2: 0.037037037037037035, 3: 0.045454545454545456, 4: 0.05, 5: 0.05, 6: 0.041666666666666664, 7: 0.041666666666666664, 8: 0.03571428571428571, 9: 0.027777777777777776}


### 圖 22-3 DataSciencter的朋友關係網路，並以不同大小來呈現不同的緊密中心度

![22-3](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-3.png?raw=true)

# 本徵向量中心度 (eigenvector centrality)

## 矩陣乘法
假設A是一個 n x m 矩陣，B是一個 m x k 矩陣。  
他們相乘後就是AB = n x k 矩陣。  
其中第(i,j)項就是：  
![formula -1](https://lh3.googleusercontent.com/xlm1AMlBJuLSdi8W6KLUIZffJOXfSDX6MaAR3YHPdD_2nojFTYsnow7ApbfFIAJfOZUJ6A7GCrLOPQwrIFfmppqWFehT9BdvoELfffHKXsNmQqkb-kt5MVjNhwBpdjex_VORIEg_lrXbrrfTIWLZWKEooQlN1vUvDs79IChnv_Zb0KFAN8PH64v55NgjeqgPLMKQ89mxFESGcqD0HfQE15TNWmNElsHImmEwRSWWReqAI5S7WsXEsoI1u_4y6FoPelIHqsxlKZS4OhyKM18uIBOzWA1cy0fO6awoqp_qdOiPYFei5w6LOKsytFuHdAuSppxO7oGhduGDPkb94YzgyXEU_vKmhwU-Hw5N-_BbUDGl-k1wwyhiVFsGGecgPMaUFVk3H8b3UfucL3G2a9Sy7LYTI5b7f4yhBzvWj63fWrASoDXsAMjv2M3SVKSp-EVjfNnnsmVkTq1B7ns6BRCw32uZbARLpkdC98bC3sueLs9t8D6d5w2OkO-QRbqkrbCKdNR9ONwYtOh0VChvmtyOoqmBVFwuadeNu3H0pdkKGk5FjG6TdGAOyXYM-2YKPF_luD3s0a9WGowfOCHdZ14_ffH2oz9EMeOdXZc0S6psc5gE2DVUhQCDP_yVDd2WkPbmas6JSFEkgomqCinyzeq_ZS0THE2IeWPAN_ewluoN-WOw69ShhJIAqSD0cWE0oQ=w360-h60-no?authuser=0)  
如果我們把m為向量當成是一個(m,1)矩陣，並與一個(n,m)的矩陣相乘。  
則會得到一個(n,1)矩陣。  
可以將它視為一個**n維向量**。



引用第四章的矩陣定義

In [ ]:
from typing import List
from typing import Callable
from typing import Tuple

Matrix=List[List[float]] #另一種型別別名

#送回一個num_rows * num_cols 的矩陣，其中第(i,j)項就是entry_fn(i,j)
def make_matrix(num_rows:int,num_cols:int,entry_fn:Callable[[int,int],float])->Matrix:

  return [[ entry_fn(i,j)        # 給定 i ，就能建立長度為 j 的列表
        for j in range(num_cols)] # [entry_fn(i,0),...]
        for i in range(num_rows)] # 針對每個i都建立一個列表

def shape(A:Matrix) -> Tuple[int,int]: # 計算矩陣的形狀(shape)

  num_rows=len(A)
  num_cols=len(A[0]) if A else 0
  return num_rows,num_cols

In [ ]:
def matrix_times_matrix(m1:Matrix,m2:Matrix) ->Matrix:
  nr1,nc1 = shaple(m1)
  nr2,nc2 = shaple(m2)
  assert nc1 == nr2 ," m1 的粽列數量，必須等於 m2 的橫行數量"

  def entry_fn(i:int,j:int) ->float:
    """m1的第i橫行與m2第j縱列的點積"""
    return sum(m1[i][k]*m2[k][j] for k in range(nc1))
  
  return make_matrix(nr1,nc2,entry_fn)


計算相乘後每個獨立的n維向量

In [ ]:
Vector = List[float]

def dot(v:Vector,w:Vector)->float:
  #計算v_1*w_1+... +v_n*w_n
  assert len(v)==len(w),"兩個向量必須有相同的維度"

  return sum(v_i*w_i for v_i,w_i in zip(v,w))

In [ ]:
def matrix_times_vector(m:Matrix,v:Vector) ->Vector:
  nr,nc = shape(m)
  n = len(v)
  assert nc == n ,"m的縱列數量必須等於v的元素數量"

  return [dot(row,v) for row in m] # 輸出長度為nr

如果A是個方矩陣(square matrix),這個運算就可以把某個n維向量,映射到另一個 維向量。有時候某個矩陣A和向量v在進行矩陣運算之後,所得到的結果正好等於某個 純量乘以向量v的結果。也就是說,計算出來的向量,恰與指向相同的方向。如果發生 這樣的情況(而且 不是零向量的話),我們就把v稱為4的「本徵向量」。而那個純量,就是所謂的「本徵值(eigenvalue)」。

[Youtube 連結 >](https://www.youtube.com/watch?v=PFDu9oVAE-g&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab&index=14&t=807s)

如果要找出A的本徵向量,其中一種可能的做法就是先取個起始向量v,套入matrix_times_vector 進行運算,然後把結果調整成向量長度為1的單位向量,再重複同樣的動作,直到整個程序收斂為止:

In [ ]:
from typing import Tuple
import random
import math

def subtrate( v:Vector, w:Vector) -> Vector:
  assert len(v) == len(w) #兩個向量必須有相同的維度

  return [ v_i-w_i for v_i,w_i in zip(v,w)]

def sum_of_squares(v:Vector) -> float:
  return dot(v,v)

def magnitude(v:Vector)->float:
  return math.sqrt(sum_of_squares(v)) #math.sqrt 是計算平方根的一個函式

def distance(v:Vector,w:Vector) -> float:
  return magnitude(subtrate(v,w))

In [ ]:
def find_eigenvector(m:Matrix,tolerance:float = 0.00001) ->Tuple[Vector,float]:
  guess = [random.random() for _ in m]

  while True:
    result = matrix_times_vector(m,guess)   # 以轉換結果為猜測值
    norm = magnitude(result)          # 計算範數
    next_guess = [x/norm for x in result] # 重新調整尺度

    if distance(guess,next_guess) < tolerance:
      # 收斂到一定程度就返回(本徵向量，特徵值)
      return next_guess,norm

    guess = next_guess

依據推測結果：  
將它套入 matrix_times_vector 進行運算，
再把結果調整成長度為1的單位向量

## 中心度

使用**相鄰矩陣(adjacency_matrix)**來表示網絡中的連結關係
第(i,j)項的值：使用者i和j是朋友，  
  
  
| Select|0|1|2|3|4|5|6|7|8|9|
| ---- |-|-|-|-|-|-|-|-|-|-|
| 0 |0|1|1|0|0|0|0|0|0|0|
| 1 |1|0|1|1|0|0|0|0|0|0|
| 2 |1|1|0|1|0|0|0|0|0|0|
| 3 |0|1|1|0|1|0|0|0|0|0|
| 4 |0|0|0|1|0|1|0|0|0|0|
| 5 |0|0|0|0|1|0|1|1|0|0|
| 6 |0|0|0|0|0|0|1|0|1|0|
| 7 |0|0|0|0|0|0|1|0|1|0|
| 8 |0|0|0|0|0|0|1|1|0|1|
| 9 |0|0|0|0|0|0|0|0|0|1|

In [ ]:
def entry_fn(i:int,j:int):
  return 1 if (i,j) in friend_pairs or (j,i) in friend_pairs else 0

In [ ]:
n = len(users)
adjacency_matrix = make_matrix(n,n,entry_fn)
print(adjacency_matrix)

[[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]


**每個節點的本徵向量中心度**  
* 中心度較高者：本身擁有較多的連結  
* 與其連結者：亦擁有較高的本徵向量中心度 

In [ ]:
eigenvector_centralities,_ = find_eigenvector(adjacency_matrix)
print(eigenvector_centralities)

[0.3857809884374945, 0.5147897597822261, 0.5147897597822261, 0.4733141283963249, 0.2336072664716933, 0.15014917341331124, 0.08355124131021355, 0.08355124131021355, 0.07284324326910803, 0.027292564982758474]


### 圖 22-4 DataSciencter的朋友關係網路，並以不同大小來呈現不同的本徵向量中心度
![22-4](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-4.jpg?raw=true)


### 為何本徵向量中心度能作為中心度的一個合理衡量方式
能計算出連結到使用者i的所有使用者相應的本徵向量中心度之總和

# 有向圖與網頁排名

## 有向圖
節點之間的單向關係

### 圖 22-5 DataSciencter的認同關係網路
![22-6](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-6.jpg?raw=true)

In [ ]:
endorsements = [ (0,1),(1,0),(0,2),(2,0),(1,2),
          (2,1),(1,3),(2,3),(3,4),(5,4),
          (5,6),(7,5),(6,8),(8,7),(8,9)]

In [ ]:
from collections import Counter

endorsement_counts = Counter(target for source,target in endorsements)

缺點：不易呈現出真正的關係，易被惡意作弊混淆結果

## 網頁排名(Page Rank) →本徵向量中心度的背後概念
關心節點之間與彼此的連接關係  
  
Google的評級方式：  
哪些網站會連結到該網站，而又有哪些網站會連結到那網站(以此類堆)

1. 網路中網頁排名的分數 總和為1.0(或是100%)。

2. 一開始,先把網頁排名的分數平均分給每個節點。

3. 在每一個步驟中,把每個節點的網頁排名分數,其中一大比例的部分,平均分配給該節點向外連結的節點。

4. 在每一個步驟中,把每個節點的網頁排名分數,其中沒分配出去的部分,平均分配給所有的節點。

In [ ]:
import tqdm

def page_rank(users:List[User],endorsements:List[Tuple[int,int]],damping:float=0.85,num_iters:int=100) ->Dict[int,float]:
  # 計算每個人認同的人數
  outgoing_counts = Counter(target for source,target in endorsements)

  # 一開始先平均分配網頁排名分數
  num_users = len(users)
  pr = {user.id:1/num_users for user in users}

  # 每次迭代，每個節點都會得到網頁排名分數的其中的一小部分
  base_pr = (1-damping)/num_users

  for iter in tqdm.trange(num_iters):
    next_pr = {user.id:base_pr for user in users} #

    for source,target in endorsements:
      # 從source 的 pr分數，取其中 damped 比例加到target中
      next_pr[target] += damping * pr[source]/outgoing_counts[source]

    pr = next_pr
  
  return pr

In [ ]:
pr = page_rank(users,endorsements)
print()
print(pr)

# Thor(編號為4)的排名分數特別高於其他人
assert pr[4] > max(page_rank for user_id,page_rank in pr.items() if user_id != 4)

100%|██████████| 100/100 [00:00<00:00, 9697.36it/s]


{0: 0.1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.14250000000000002, 5: 0.1, 6: 0.1, 7: 0.1, 8: 0.1, 9: 0.1}


### 圖 22-6 DataSciencter的認同關係網路，並以不同大小來呈現不同的網頁排名分數
![22-9](https://github.com/hank199599/data_science_from_scratch_reading_log/blob/main/pictures/22-9.jpg?raw=true)